In [72]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [73]:
ex = pd.read_json('support_files/exercises-concepts.json')

In [74]:
ex

,id,name,ordern,description,image,total_levels,type,folder,video,disabled
0,1,Fun with Basics,1,"Sequence, Algorithmic Thinking",fun_with_basics.png,10,maze,fun_with_basics,FWB_LI FINAL.mp4,0
1,2,Loopy Loops,2,"Loops, Debugging",loopy_loops.png,12,maze,loopy_loops,,0
2,3,Conditional Crops,3,"Conditional Statements, Pattern Recognition",conditional_crops.png,12,maze,conditional_crops,,0
3,4,Backyard Functions,4,"Functions, Variables, Events",backyard_functions.png,10,maze,backyard_functions,,0
4,5,Dog and the loops,5,"Loops, Variables, Functions",Dog and the loops - Grade 4.png,8,maze,dog_and_the_loops,,0
5,6,Gardening Conditionals,6,"Functions, Conditional Statements, Sequence, A...",Gardening Conditionals - Grade 4.png,6,maze,gardening_conditionals,,0
6,7,Swamp conditionals,7,"Conditional Statements, Loops, Variables, Sequ...",Swamp Conditionals - Grade 4.png,4,maze,swamp_conditionals,,0
7,8,Baloon pop functions,8,"Conditional Statements, Loops, Variables, Sequ...",Baloon pop functions - Grade 4.png,8,maze,baloon_pop_functions,,0
8,9,Loops and castles,9,"Loops, Variables, Functions",Loops and Castles - Grade 5.png,8,maze,loops_and_castles,,0
9,10,Desert conditionals,10,"Conditional Statements, Loops, Variables, Sequ...",Desert Conditionals - Grade 5.png,4,maze,desert_conditionals,,0


In [75]:
total_exercise_count = ex['id'].count()
total_level_count = ex['total_levels'].sum()

In [76]:
df = pd.read_json('support_files/student_report_itk.json')

In [77]:
df.shape

(42347, 11)

In [78]:
df.head()

,user_id,name,school,grade,division,exercise_id,exercise_name,level,total_levels,level_completion_status,blocks
0,7695,Eva Admin,,,,13,Introduction to Gaming - Grade 5,1,3,0,"<xml xmlns=""https://developers.google.com/bloc..."
1,7695,Eva Admin,,,,13,Introduction to Gaming - Grade 5,2,3,0,
2,1,,,100,A,1,Fun with Basics,1,10,1,"<xml xmlns=""https://developers.google.com/bloc..."
3,1,,,100,A,2,Loopy Loops,1,12,0,
4,1,,,100,A,1,Fun with Basics,2,10,1,"<xml xmlns=""https://developers.google.com/bloc..."


In [79]:
df = df[~(df['grade']=='')]

In [80]:
df.head()

,user_id,name,school,grade,division,exercise_id,exercise_name,level,total_levels,level_completion_status,blocks
2,1,,,100,A,1,Fun with Basics,1,10,1,"<xml xmlns=""https://developers.google.com/bloc..."
3,1,,,100,A,2,Loopy Loops,1,12,0,
4,1,,,100,A,1,Fun with Basics,2,10,1,"<xml xmlns=""https://developers.google.com/bloc..."
5,1,,,100,A,1,Fun with Basics,3,10,1,"<xml xmlns=""https://developers.google.com/bloc..."
6,1,,,100,A,1,Fun with Basics,4,10,0,


In [81]:
def count_blocks(xml):
    soup = BeautifulSoup(xml, 'html.parser')
    return len(soup.find_all('block'))

In [82]:
df['block_count'] = df['blocks'].apply(count_blocks)

In [83]:
def proper_case(string):
    parts = string.split(' ')

    return ' '.join(word.title() for word in parts[0:])

In [84]:
df.head()

,user_id,name,school,grade,division,exercise_id,exercise_name,level,total_levels,level_completion_status,blocks,block_count
2,1,,,100,A,1,Fun with Basics,1,10,1,"<xml xmlns=""https://developers.google.com/bloc...",4
3,1,,,100,A,2,Loopy Loops,1,12,0,,0
4,1,,,100,A,1,Fun with Basics,2,10,1,"<xml xmlns=""https://developers.google.com/bloc...",6
5,1,,,100,A,1,Fun with Basics,3,10,1,"<xml xmlns=""https://developers.google.com/bloc...",9
6,1,,,100,A,1,Fun with Basics,4,10,0,,0


In [85]:
df['name'] = df['name'].apply(proper_case)

In [86]:
df_ex_completed = df[df['level_completion_status']==1]

In [87]:
df_ex_completed.shape

(30909, 12)

In [88]:
df_ex_completed

,user_id,name,school,grade,division,exercise_id,exercise_name,level,total_levels,level_completion_status,blocks,block_count
2,1,,,100,A,1,Fun with Basics,1,10,1,"<xml xmlns=""https://developers.google.com/bloc...",4
4,1,,,100,A,1,Fun with Basics,2,10,1,"<xml xmlns=""https://developers.google.com/bloc...",6
5,1,,,100,A,1,Fun with Basics,3,10,1,"<xml xmlns=""https://developers.google.com/bloc...",9
110,7714,Aabha Patil,Bhiwandi School,1,Sapphire,1,Fun with Basics,1,10,1,"<xml xmlns=""https://developers.google.com/bloc...",3
111,7714,Aabha Patil,Bhiwandi School,1,Sapphire,45,Fun with Basics,1,3,1,"<xml xmlns=""https://developers.google.com/bloc...",10
...,...,...,...,...,...,...,...,...,...,...,...,...
42339,7880,Owais Mehbub Shaikh,SKS,8,Turquoise,11,Predator bird functions,7,7,1,"<xml xmlns=""https://developers.google.com/bloc...",22
42340,7880,Owais Mehbub Shaikh,SKS,8,Turquoise,12,Functions on the field,7,9,1,,0
42341,7880,Owais Mehbub Shaikh,SKS,8,Turquoise,5,Dog and the loops,8,8,1,"<xml xmlns=""https://developers.google.com/bloc...",11
42343,7880,Owais Mehbub Shaikh,SKS,8,Turquoise,12,Functions on the field,8,9,1,,0


# BLOCK 1 

Levels Completed | School Average | Exercises Completed | School Average

In [89]:
report_block1 = df_ex_completed.groupby(['user_id', 'name', 'grade', 'division']).agg(
    exercises_done=('exercise_id', 'nunique'),
    levels_done=('level', 'nunique')).reset_index()

In [90]:
report_block1['ex_completion_rate'] = (report_block1['exercises_done']/total_exercise_count).round(2)
report_block1['lev_completion_rate'] = (report_block1['levels_done']/total_level_count).round(2)

In [91]:
report_block1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              1969 non-null   int64  
 1   name                 1969 non-null   object 
 2   grade                1969 non-null   object 
 3   division             1969 non-null   object 
 4   exercises_done       1969 non-null   int64  
 5   levels_done          1969 non-null   int64  
 6   ex_completion_rate   1969 non-null   float64
 7   lev_completion_rate  1969 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 123.2+ KB


In [92]:
df_wo_duplicates = df.drop_duplicates(subset=['user_id']).reset_index()

In [93]:
final_report_block1 = pd.merge(report_block1, df_wo_duplicates[['user_id', 'school']], on='user_id', how='left')

In [94]:
final_report_block1.shape

(1969, 9)

In [95]:
final_report_block1.shape

(1969, 9)

In [96]:
average_exercise = final_report_block1.groupby(['school'])['exercises_done'].mean().round(1)
average_exercise_dict = dict(zip(average_exercise.index, average_exercise.values))

In [97]:
average_exercise_dict

{'': 1.0,
 'Bhiwandi School': 3.1,
 'DWPS': 2.5,
 'Eva Indore': 3.2,
 'Eva Thane': 3.0,
 'Eva World School': 2.5,
 'IRA': 2.3,
 'SKS': 2.5}

In [98]:
final_report_block1['school_ex_avg'] = final_report_block1['school'].map(average_exercise_dict)

In [99]:
average_level = final_report_block1.groupby('school')['levels_done'].mean().round(2)
average_level_dict = dict(zip(average_level.index, average_level.values))

In [100]:
average_level_dict

{'': 3.0,
 'Bhiwandi School': 8.04,
 'DWPS': 8.26,
 'Eva Indore': 9.78,
 'Eva Thane': 8.1,
 'Eva World School': 6.52,
 'IRA': 7.34,
 'SKS': 7.74}

In [101]:
final_report_block1['school_lev_avg'] = final_report_block1['school'].map(average_level_dict)

In [102]:
final_report_block1.head()

,user_id,name,grade,division,exercises_done,levels_done,ex_completion_rate,lev_completion_rate,school,school_ex_avg,school_lev_avg
0,1,,100,A,1,3,0.02,0.01,,1.0,3.00
1,4682,A Isha Angel,3,Garnet,3,6,0.06,0.02,Eva World School,2.5,6.52
2,4683,A Joy Samuel,2,Zircon,2,1,0.04,0.00,Eva World School,2.5,6.52
3,4684,Aabha Siddhartha Chandurkar,3,Aquamarine,2,2,0.04,0.01,Eva World School,2.5,6.52
4,4685,Aadit Jignesh Limbani,1,Sapphire,5,10,0.10,0.03,Eva World School,2.5,6.52


In [103]:
final_report_block1.to_csv('support_files/report_block1.csv')

# BLOCK 2

Exercise Name | Total Levels | Levels Completed | Concepts | Blocks Used

In [104]:
df_ex_completed.head()

,user_id,name,school,grade,division,exercise_id,exercise_name,level,total_levels,level_completion_status,blocks,block_count
2,1,,,100,A,1,Fun with Basics,1,10,1,"<xml xmlns=""https://developers.google.com/bloc...",4
4,1,,,100,A,1,Fun with Basics,2,10,1,"<xml xmlns=""https://developers.google.com/bloc...",6
5,1,,,100,A,1,Fun with Basics,3,10,1,"<xml xmlns=""https://developers.google.com/bloc...",9
110,7714,Aabha Patil,Bhiwandi School,1,Sapphire,1,Fun with Basics,1,10,1,"<xml xmlns=""https://developers.google.com/bloc...",3
111,7714,Aabha Patil,Bhiwandi School,1,Sapphire,45,Fun with Basics,1,3,1,"<xml xmlns=""https://developers.google.com/bloc...",10


In [105]:
report_block2 = df_ex_completed.groupby(['user_id', 'name', 'exercise_id', 'exercise_name', 'total_levels']).agg(levels_done=('level', 'nunique'), blocks_used=('block_count', 'sum')).reset_index()

In [106]:
report_block2.shape

(5113, 7)

In [107]:
report_block2

,user_id,name,exercise_id,exercise_name,total_levels,levels_done,blocks_used
0,1,,1,Fun with Basics,10,3,19
1,4682,A Isha Angel,1,Fun with Basics,10,5,43
2,4682,A Isha Angel,2,Loopy Loops,12,1,5
3,4682,A Isha Angel,8,Baloon pop functions,8,1,0
4,4683,A Joy Samuel,1,Fun with Basics,10,1,6
...,...,...,...,...,...,...,...
5108,7959,Sidra,7,Swamp conditionals,4,4,187
5109,7995,Aadidev Arun,1,Fun with Basics,10,4,26
5110,7996,Anuraj Singh,49,Fun with Basics - Grade 1 & 2,8,8,0
5111,7996,Anuraj Singh,50,Loopy Loops - Grade 1/2,8,8,0


In [108]:
report_block2[['exercise_name', 'total_levels']].value_counts()

exercise_name                  total_levels
Fun with Basics                10              1327
Loopy Loops                    12               874
Fun with Basics - Grade 1 & 2  8                738
Loopy Loops - Grade 1/2        8                610
Dog and the loops              8                447
Conditional Crops              12               261
Swamp conditionals             4                222
Baloon pop functions           8                116
Fun with Basics                3                102
Desert conditionals            4                 91
Loops and castles              8                 74
Predator bird functions        7                 69
Functions on the field         9                 67
Backyard Functions             10                55
Loopy Loops                    4                 46
Backyard Functions             6                 10
Gardening Conditionals         6                  4
dtype: int64

# Block 3

Concepts | Explanation

In [109]:
ex.rename(columns={'id': 'exercise_id'}, inplace=True)

In [110]:
ex.head()

,exercise_id,name,ordern,description,image,total_levels,type,folder,video,disabled
0,1,Fun with Basics,1,"Sequence, Algorithmic Thinking",fun_with_basics.png,10,maze,fun_with_basics,FWB_LI FINAL.mp4,0
1,2,Loopy Loops,2,"Loops, Debugging",loopy_loops.png,12,maze,loopy_loops,,0
2,3,Conditional Crops,3,"Conditional Statements, Pattern Recognition",conditional_crops.png,12,maze,conditional_crops,,0
3,4,Backyard Functions,4,"Functions, Variables, Events",backyard_functions.png,10,maze,backyard_functions,,0
4,5,Dog and the loops,5,"Loops, Variables, Functions",Dog and the loops - Grade 4.png,8,maze,dog_and_the_loops,,0


In [111]:
report_block2 = pd.merge(report_block2, ex[['description', 'exercise_id']], on='exercise_id', how='left')

In [112]:
columns = report_block2.columns.tolist()
columns

['user_id',
 'name',
 'exercise_id',
 'exercise_name',
 'total_levels',
 'levels_done',
 'blocks_used',
 'description']

In [113]:
report_block2 = report_block2[['user_id',
 'name',
 'exercise_id',
 'exercise_name',
 'description',
 'total_levels',
 'levels_done',
 'blocks_used']]

In [114]:
report_block2.head()

,user_id,name,exercise_id,exercise_name,description,total_levels,levels_done,blocks_used
0,1,,1,Fun with Basics,"Sequence, Algorithmic Thinking",10,3,19
1,4682,A Isha Angel,1,Fun with Basics,"Sequence, Algorithmic Thinking",10,5,43
2,4682,A Isha Angel,2,Loopy Loops,"Loops, Debugging",12,1,5
3,4682,A Isha Angel,8,Baloon pop functions,"Conditional Statements, Loops, Variables, Sequ...",8,1,0
4,4683,A Joy Samuel,1,Fun with Basics,"Sequence, Algorithmic Thinking",10,1,6


In [115]:
report_block2.to_csv('support_files/report_block2.csv')

In [116]:
import json
with open ('support_files/concepts.json', 'r') as file:
    conceptsJSON = json.load(file)

In [117]:
conceptsJSON['Computational_Thinking_Concepts']['Algorithmic Thinking']

'Developing step-by-step instructions or rules to solve a problem.'

In [118]:
df[df['name']=='Raj Jadhav']

,user_id,name,school,grade,division,exercise_id,exercise_name,level,total_levels,level_completion_status,blocks,block_count
383,7706,Raj Jadhav,Bhiwandi School,1,Sapphire,1,Fun with Basics,1,10,1,"<xml xmlns=""https://developers.google.com/bloc...",3
384,7706,Raj Jadhav,Bhiwandi School,1,Sapphire,2,Loopy Loops,1,12,1,"<xml xmlns=""https://developers.google.com/bloc...",4
385,7706,Raj Jadhav,Bhiwandi School,1,Sapphire,49,Fun with Basics - Grade 1 & 2,1,8,1,,0
386,7706,Raj Jadhav,Bhiwandi School,1,Sapphire,50,Loopy Loops - Grade 1/2,1,8,1,,0
387,7706,Raj Jadhav,Bhiwandi School,1,Sapphire,1,Fun with Basics,2,10,1,"<xml xmlns=""https://developers.google.com/bloc...",1
388,7706,Raj Jadhav,Bhiwandi School,1,Sapphire,2,Loopy Loops,2,12,0,"<xml xmlns=""https://developers.google.com/bloc...",2
389,7706,Raj Jadhav,Bhiwandi School,1,Sapphire,49,Fun with Basics - Grade 1 & 2,2,8,1,,0
390,7706,Raj Jadhav,Bhiwandi School,1,Sapphire,50,Loopy Loops - Grade 1/2,2,8,1,,0
391,7706,Raj Jadhav,Bhiwandi School,1,Sapphire,1,Fun with Basics,3,10,1,"<xml xmlns=""https://developers.google.com/bloc...",9
392,7706,Raj Jadhav,Bhiwandi School,1,Sapphire,49,Fun with Basics - Grade 1 & 2,3,8,1,,0


In [121]:
final_report_block1['grade'].value_counts()

1      387
3      349
2      334
4      289
5      176
6      106
7      102
8       70
II      30
10      29
9       20
III     19
VII     18
I       12
IV      11
VI      10
V        6
100      1
Name: grade, dtype: int64

# No Exercise Done Report

In [176]:
df_na = pd.read_json('support_files/student_report_itk_na.json')

In [177]:
df_na = df_na[~(df_na['grade']=='')]

In [178]:
df_na['name'] = df_na['name'].apply(proper_case)

In [179]:
df_na.head()

,user_id,name,school,grade,division
3,7742,Reyansh Ojha,Bhiwandi School,2,Zircon
4,8004,Adi Hariya,Bhiwandi School,6,Turquoise
5,8003,Ayush Choudhary,Bhiwandi School,7,Amethyst
6,6138,Kshitij Ravi Divakar,DWPS,1,Sapphire
7,6178,Navika Naveen Poojary,DWPS,1,Sapphire


In [180]:
report_block1_na = df_na

In [182]:
additional_columns = ['exercises_done', 'levels_done',
       'ex_completion_rate', 'lev_completion_rate']

for column in additional_columns:
    report_block1_na[column] = 0

In [197]:
report_block1_na = report_block1_na[['user_id', 'name', 'grade', 'division', 'exercises_done',
       'levels_done', 'ex_completion_rate', 'lev_completion_rate', 'school',
       'school_ex_avg', 'school_lev_avg']]

In [183]:
report_block1_na['school_ex_avg'] = report_block1_na['school'].map(average_exercise_dict)

In [184]:
report_block1_na['school_lev_avg'] = report_block1_na['school'].map(average_level_dict)

In [198]:
report_block1_na.head()

,user_id,name,grade,division,exercises_done,levels_done,ex_completion_rate,lev_completion_rate,school,school_ex_avg,school_lev_avg
3,7742,Reyansh Ojha,2,Zircon,0,0,0,0,Bhiwandi School,3.1,8.04
4,8004,Adi Hariya,6,Turquoise,0,0,0,0,Bhiwandi School,3.1,8.04
5,8003,Ayush Choudhary,7,Amethyst,0,0,0,0,Bhiwandi School,3.1,8.04
6,6138,Kshitij Ravi Divakar,1,Sapphire,0,0,0,0,DWPS,2.5,8.26
7,6178,Navika Naveen Poojary,1,Sapphire,0,0,0,0,DWPS,2.5,8.26


In [199]:
report_block1_na.to_csv('support_files/report_block1_na.csv')

In [188]:
report_block2_na = df_na[['user_id', 'name']]

In [189]:
report_block2_na.head()

,user_id,name
3,7742,Reyansh Ojha
4,8004,Adi Hariya
5,8003,Ayush Choudhary
6,6138,Kshitij Ravi Divakar
7,6178,Navika Naveen Poojary


In [204]:
addtional_columns_block2_1 = ['exercise_id', 'exercise_name', 'description']

for column in addtional_columns_block2_1:
    report_block2_na[column] = 'None'

addtional_columns_block2_2 = ['total_levels', 'levels_done', 'blocks_used']
for column in addtional_columns_block2_2:
    report_block2_na[column] = 0

/var/folders/2f/qrm1hg3945j9tqtn7tmw3vq40000gn/T/ipykernel_22873/2887905055.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_block2_na[column] = 'None'
/var/folders/2f/qrm1hg3945j9tqtn7tmw3vq40000gn/T/ipykernel_22873/2887905055.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_block2_na[column] = 0


In [205]:
report_block2_na.head()

,user_id,name,exercise_id,exercise_name,description,total_levels,levels_done,blocks_used
3,7742,Reyansh Ojha,None,None,None,0,0,0
4,8004,Adi Hariya,None,None,None,0,0,0
5,8003,Ayush Choudhary,None,None,None,0,0,0
6,6138,Kshitij Ravi Divakar,None,None,None,0,0,0
7,6178,Navika Naveen Poojary,None,None,None,0,0,0


In [206]:
report_block2_na.to_csv('support_files/report_block2_na.csv')